In [2]:
import pandas as pd
import numpy as np
pd.set_option('max_columns', 50)
from datetime import datetime
import matplotlib as mpl
mpl.rcParams['lines.linewidth'] = 2

In [3]:
df = pd.read_csv('Data Set 2.csv')
df.head() 

,OrderId,CustomerId,LocationId,channel,Amount,OrderDate,BillNo
0,35623100,6662,4416,foodpanda,267,1/1/2018,DEL
1,35622814,8237,4002,zomato,1051,1/1/2018,DEL
2,35623284,11912,4416,takeaway,714,1/1/2018,DEL
3,35631171,14563,218,zomato,834,1/1/2018,GGN
4,35635515,14833,295,call,544,1/1/2018,DEL


In [4]:
df = df[df['BillNo'] == 'DEL']

In [5]:
df['OrderDate']= pd.to_datetime(df['OrderDate'])

In [6]:
df['OrderPeriod'] = df.OrderDate.apply(lambda x: x.strftime('%Y-%m'))

In [7]:
df.head()

,OrderId,CustomerId,LocationId,channel,Amount,OrderDate,BillNo,OrderPeriod
0,35623100,6662,4416,foodpanda,267,2018-01-01,DEL,2018-01
1,35622814,8237,4002,zomato,1051,2018-01-01,DEL,2018-01
2,35623284,11912,4416,takeaway,714,2018-01-01,DEL,2018-01
4,35635515,14833,295,call,544,2018-01-01,DEL,2018-01
5,35622397,18286,4002,zomato,613,2018-01-01,DEL,2018-01


In [8]:
#Revenue
RevByMonth = df.groupby('OrderPeriod')['Amount'].sum() 
RevByMonth = pd.DataFrame(RevByMonth)
RevByMonth.head()

,Amount
OrderPeriod,
2018-01,5708169
2018-02,5579848
2018-03,5970987
2018-04,6038631
2018-05,6975206


In [9]:
#Orders
OrderByMonth = df.groupby('OrderPeriod')['OrderId'].count() #Orders
OrderByMonth = pd.DataFrame(OrderByMonth)
OrderByMonth.head()

,OrderId
OrderPeriod,
2018-01,9838
2018-02,9979
2018-03,10529
2018-04,10631
2018-05,12804


In [10]:
#Customers
CusByMonth = df.groupby("OrderPeriod").agg({"CustomerId": pd.Series.nunique}) 
CusByMonth.head()

,CustomerId
OrderPeriod,
2018-01,7269
2018-02,7359
2018-03,7620
2018-04,7863
2018-05,9131


In [11]:
#Repeat Customers
grouped = df.groupby(['OrderPeriod', 'CustomerId'])
RC = grouped.agg({'OrderId': pd.Series.nunique})
RC.head()

OrderId
OrderPeriod CustomerId         
2018-01     134               1
            1541              1
            2087              1
            3811              1
            4476              1

In [12]:
RCNew = RC[RC.OrderId > 1]
RCNew.head()

OrderId
OrderPeriod CustomerId         
2018-01     4593              4
            6172              2
            6751              2
            8237              4
            9774              2

In [13]:
RepCustomers = RCNew.groupby('OrderPeriod').count() 
RepCustomers.rename(columns={'OrderId': 'RepeatCustomers'}, inplace=True)
RepCustomers.head()

,RepeatCustomers
OrderPeriod,
2018-01,1467
2018-02,1550
2018-03,1694
2018-04,1622
2018-05,2192


In [14]:
#New Customers
df.set_index('CustomerId', inplace=True)
df['CohortGroup'] = df.groupby(level=0)['OrderDate'].min().apply(lambda x: x.strftime('%Y-%m'))
df.reset_index(inplace=True)
df.head()

,CustomerId,OrderId,LocationId,channel,Amount,OrderDate,BillNo,OrderPeriod,CohortGroup
0,6662,35623100,4416,foodpanda,267,2018-01-01,DEL,2018-01,2018-01
1,8237,35622814,4002,zomato,1051,2018-01-01,DEL,2018-01,2018-01
2,11912,35623284,4416,takeaway,714,2018-01-01,DEL,2018-01,2018-01
3,14833,35635515,295,call,544,2018-01-01,DEL,2018-01,2018-01
4,18286,35622397,4002,zomato,613,2018-01-01,DEL,2018-01,2018-01


In [15]:
grouped = df.groupby(['CohortGroup', 'OrderPeriod'])
newC = grouped.agg({'CustomerId': pd.Series.nunique,
                       'OrderId': pd.Series.nunique})
newC.head()

CustomerId  OrderId
CohortGroup OrderPeriod                     
2018-01     2018-01            7269     9838
            2018-02            1520     2829
            2018-03            1333     2502
            2018-04            1138     2189
            2018-05            1033     2053

In [16]:
def cohort_period(df):
    
    df['CohortPeriod'] = np.arange(len(df)) + 1
    return df

newC = newC.groupby(level=0).apply(cohort_period)
newC.head()

CustomerId  OrderId  CohortPeriod
CohortGroup OrderPeriod                                   
2018-01     2018-01            7269     9838             1
            2018-02            1520     2829             2
            2018-03            1333     2502             3
            2018-04            1138     2189             4
            2018-05            1033     2053             5

In [17]:
NewCustomers = newC[newC['CohortPeriod'] == 1]

In [18]:
NC = NewCustomers.reset_index()

In [19]:
NC = NC.drop(['CohortGroup'], axis =1)

In [20]:
NC = NC.set_index('OrderPeriod')

In [21]:
NC = NC[['CustomerId']]

In [22]:
NC.rename(columns={'CustomerId': 'NewCustomers'}, inplace=True)

In [23]:
NC.head()

,NewCustomers
OrderPeriod,
2018-01,7269
2018-02,5839
2018-03,5477
2018-04,5381
2018-05,6430


In [24]:
dfMerge = pd.concat([RevByMonth, OrderByMonth, CusByMonth, NC,RepCustomers],axis=1,sort=False).reset_index()
dfMerge.rename(columns = {'index':'OrderPeriod'})
dfMerge.rename(columns={'Amount': 'Revenue',
                        'OrderId': 'Orders',
                        'CustomerId':'Customers'}, inplace=True)

In [25]:
dfMerge['Frequency'] = dfMerge['Orders']/dfMerge['Customers']

In [26]:
dfMerge = dfMerge.set_index('OrderPeriod')

In [27]:
dfMerge.transpose()

OrderPeriod,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04
Revenue,5.708169e+06,5.579848e+06,5.970987e+06,6.038631e+06,6.975206e+06,8.006836e+06,7.323943e+06,6.147812e+06,6.420724e+06,9.113816e+06,1.260435e+07,1.043584e+07,7.255724e+06,8.019485e+06,1.048652e+07,2.792814e+06
Orders,9.838000e+03,9.979000e+03,1.052900e+04,1.063100e+04,1.280400e+04,1.437100e+04,1.325400e+04,1.108100e+04,1.264600e+04,1.539600e+04,2.134600e+04,1.718300e+04,1.284400e+04,1.720200e+04,2.464000e+04,7.383000e+03
Customers,7.269000e+03,7.359000e+03,7.620000e+03,7.863000e+03,9.131000e+03,1.026900e+04,1.009500e+04,8.671000e+03,9.937000e+03,1.176900e+04,1.574500e+04,1.333300e+04,9.960000e+03,1.349600e+04,1.856000e+04,6.165000e+03
NewCustomers,7.269000e+03,5.839000e+03,5.477000e+03,5.381000e+03,6.430000e+03,7.364000e+03,7.204000e+03,5.819000e+03,7.062000e+03,8.834000e+03,1.158200e+04,8.909000e+03,6.132000e+03,8.741000e+03,1.228600e+04,3.486000e+03
RepeatCustomers,1.467000e+03,1.550000e+03,1.694000e+03,1.622000e+03,2.192000e+03,2.608000e+03,2.068000e+03,1.581000e+03,1.760000e+03,2.216000e+03,3.395000e+03,2.460000e+03,1.863000e+03,2.397000e+03,3.650000e+03,8.590000e+02
Frequency,1.353419e+00,1.356027e+00,1.381759e+00,1.352028e+00,1.402256e+00,1.399455e+00,1.312927e+00,1.277938e+00,1.272617e+00,1.308183e+00,1.355732e+00,1.288757e+00,1.289558e+00,1.274600e+00,1.327586e+00,1.197567e+00


In [28]:
dfMerge.to_excel(r'C:\Users\user\Desktop\Xeno\Del Report.xlsx')